## Character-level LSTM Generation

In [2]:
# load Nietzsche text
import tensorflow.keras
import numpy as np

path = tensorflow.keras.utils.get_file(
        'nietzche.txt',
        origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus lenght:', len(text))

corpus lenght: 600893


In [3]:
# vectorize text into one-hot overlapping sequences and next-char target
max_len = 60
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - max_len, 3):
    sentences.append(text[i : i + max_len])
    next_chars.append(text[i+max_len])
print('number of sequences:', len(sentences))

chars = sorted(list(set(text)))
chars_dict = dict((char, chars.index(char)) for char in chars)
print('number of unique chars:', len(chars))

print('Vectorizing text...')
x = np.zeros((len(sentences), max_len, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for char_i, c in enumerate(sentence):
        x[i, char_i, chars_dict[c]] = 1
    y[i, chars_dict[next_chars[i]]] = 1
print("done")

number of sequences: 200278
number of unique chars: 57
Vectorizing text...
done


In [9]:
# build LSTM network
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(max_len, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer=optimizers.legacy.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/Users/nate/miniconda3/envs/tfenv/lib/python3.9/site-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
# function to sample the next char distribution
# with custom softmax tempurature
def sample(preds, temperature=1.0):
    preds = np.array(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)
    preds /= np.sum(preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [11]:
# train model and generate texts at every epoch with different temps
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    random_seed = random.randint(0, len(text) - max_len - 1)
    generated_text = text[random_seed : random_seed + max_len]
    print('--- Generating with seed: \"' + generated_text + '\"')
    
    for temp in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temp)
        input_text = generated_text
        sys.stdout.write(input_text)
        for i in range(400):
            sampled = np.zeros((1, max_len, len(chars)))
            for char_i, c in enumerate(input_text):
                sampled[0, char_i, chars_dict[c]] = 1
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature=temp)
            next_char = chars[next_index]
            
            input_text += next_char
            input_text = input_text[1:]
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 37s 24ms/step - loss: 1.4435
--- Generating with seed: "philosophie, c'est-a-dire
pour voir clair dans ce qui est."
"
----- temperature: 0.2
philosophie, c'est-a-dire
pour voir clair dans ce qui est."

12e
toone and strength of the form of the same the sense of the for the soul of the standard of the horther of the propens the such a man of the standard of the standard and sense of the standard of the propens of the standard of the superlition and standing the sense of the sense of the fine the same such an of the superiority of the far and in the standard of the spirit would be an opinion of t
----- temperature: 0.5
philosophie, c'est-a-dire
pour voir clair dans ce qui est."

1an has to the death of the man in the conception of the standard of the such the bad the strength and strange of the problem and distincey, the contempt to restrusion and things and bad and has been such a means of the morality with the same time of the fact a

/var/folders/g9/nvr26fkn4hg3pgv39ty__9qc0000gn/T/ipykernel_65755/1930721140.py:5: RuntimeWarning: divide by zero encountered in log
  preds = np.exp(np.log(preds) / temperature)


ne another, and woman when i the pathous of all expecting and the spirit of the spirits to the case the same tartish of the fashions of the matter of the sense of the exist in the german presumption and problem of the and manifordation the learne to the self-deluse, in the stronger and more shrouked of the present the rare condition of the destiny a
----- temperature: 1.0
, indeed, that
is recognized at all, even for a god: human, european, becomes, begov of cheits,, and whom distrusts of our ish from proteibs
under
and beside--as the
egycsel--"ever in the matcided fee--as are
respectment and dissutsy decreass, asce of nature and richer, it is let us! recultor, its spare created moralist, guel,
is and contemnly, the, mitaince noced
my will to
"to those conrurioum, the sunjere from the pathous functions--iftenc
unus
exist, i
----- temperature: 1.2
, indeed, that
is recognized at all, even for a god: human, also down mightly make
insce;iblisacing
erlufriativm obtainsdiqual, wroft ba hyou